In [ ]:
# import required packages
import boto3
import json
import math
from IPython.display import Image, display
from matplotlib import pyplot as plt

# aws service client
s3_obj = boto3.client('s3')

In [ ]:
# install OpenCV and import cv2
!pip3 install opencv-python &> /dev/null
import cv2
import numpy as np

# read image from Amazon S3 as bytes format
s3_bucket_name = "nutcha-sagemaker-bucket"
image_name = "images/src_image.jpg"
response = s3_obj.get_object(
    Bucket=s3_bucket_name,
    Key=image_name
)

image_printing = []
image_raw = response['Body']
image_bytes = image_raw.read()

# convert image bytes array into numpy array
nparr = np.frombuffer(image_bytes, np.uint8)
img_origin = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
# 1st image
image_printing.append(["Original image", img_origin])
# image_printing.append(["Before", img_origin])

# make the image blur
img_blur = cv2.medianBlur(img_origin,5)
image_printing.append(["Blured image", img_blur])

# make image more cleansing
# ret,th4 = cv2.threshold(img_blur,210,255,cv2.THRESH_BINARY)
# image_printing.append(["Global Thresholding (v = 210)", th4])

# gray image
gray_img = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
image_printing.append(["Gray image", gray_img])

# make image more cleansing
# ret,th1 = cv2.threshold(gray_img,127,255,cv2.THRESH_BINARY)
# image_printing.append(["Global Thresholding (v = 127)", th1])

# th2 = cv2.adaptiveThreshold(gray_img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
# image_printing.append(["Adaptive Mean Thresholding", th2])

th3 = cv2.adaptiveThreshold(gray_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
image_printing.append(["Adaptive Gaussian Thresholding", th3])

# image_printing.append(["After", th3])

# reduce noice
# img_noiced = cv2.fastNlMeansDenoisingColored(img_np,None,20,10,7,21)
# image_printing.append(["Reduce noice", img_noiced])

# prepare to save processed image
processed_image = th3

In [ ]:
# printing images
plt.close()
figure_size = 50
plt.figure(figsize=(20,figure_size))
num_col = 2
num_row = math.ceil((len(image_printing)+1) / num_col)
image_count = 1
for image in image_printing:
    plt.subplot(num_row,num_col,image_count)
    plt.title(image[0])
    plt.imshow(image[1],cmap='gray')
    image_count=image_count+1
plt.show()

In [ ]:
# save processed image back to s3 bucket
target_s3_bucket = "nutcha-sagemaker-bucket"
target_s3_prefix = "target_images/"
original_image_name = image_name.split('/')[-1]

# convert byted image back to original image format
success, encoded_image = cv2.imencode('.jpg', processed_image)
print("Convert image status = {}".format(success))

# put image to back to s3 bucket
response = s3_obj.put_object(
    Body=encoded_image.tobytes(),
    Bucket=target_s3_bucket,
    Key=target_s3_prefix + "post_" + original_image_name
)
print("Uploaded success")